In [1]:
# !pip install jieba sentencepiece transformers

In [2]:
import os
import pickle
import random

import numpy as np
from tqdm import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import T5ForConditionalGeneration
from tokenization_enc_dec import EncDecTokenizer

In [3]:
class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, batch_size = 16, max_len=50):
        super(Dataset).__init__()
        self.batch_size = batch_size
        
        data = []
        data_root = '../chatdata/'
        for f in os.listdir(data_root):
            data += pickle.load(open(os.path.join(data_root, f), 'rb'))
        self.data = data

    def random_data(self):
        one_data = random.choice(self.data)
        lens = random.randint(2, 5)
        ind = random.randint(0, len(one_data) - lens - 1)
        sentences = one_data[ind:ind+lens]
        maxlen = max([len(x) for x in sentences])
        if maxlen > 40:
            return self.random_data()
        s = ''.join(sentences)
        if '拜拜' in s or '再见' in s:
            if random.random() < 0.95:
                return self.random_data()
        return sentences

    def get_single_data(self, sentences):
        input_ids = []
        for s in sentences[:-1]:
            input_ids += tokenizer.encode(s) + [tokenizer.sep_id]
        input_ids += [tokenizer.get_sentinel_id(0)]
        y_ids = tokenizer.encode(sentences[-1])
        decoder_input_ids = [tokenizer.get_sentinel_id(0)] + y_ids
        labels = y_ids + [tokenizer.sep_id]
        return torch.LongTensor(input_ids), torch.LongTensor(decoder_input_ids), torch.LongTensor(labels)
        
    def __iter__(self):
        batch_size = self.batch_size
        batch = []
        keep = np.random.randint(2, 6)
        while True:
            ids, dids, lbl = self.get_single_data(self.random_data())
            batch.append((ids, dids, lbl))
            if len(batch) >= batch_size:
                input_ids = pad_sequence([
                    x[0]
                    for x in batch
                ], batch_first=True, padding_value=tokenizer.pad_id)
                mask = (input_ids != tokenizer.pad_id).to(input_ids.dtype)
                decoder_input_ids = pad_sequence([
                    x[1]
                    for x in batch
                ], batch_first=True, padding_value=tokenizer.pad_id)
                decoder_mask = (decoder_input_ids != tokenizer.pad_id).to(input_ids.dtype)
                # padding -100是源代码里面的magic number， 参考：
                # https://github.com/huggingface/transformers/blob/1c06240e1b3477728129bb58e7b6c7734bb5074e/src/transformers/models/t5/modeling_t5.py#L1580
                labels = pad_sequence([
                    x[2]
                    for x in batch
                ], batch_first=True, padding_value=-100)
                yield input_ids, mask, decoder_input_ids, decoder_mask, labels
                batch = []
                keep = np.random.randint(2, 6)

In [4]:
%%time
model = T5ForConditionalGeneration.from_pretrained('./torch_eva/')
tokenizer = EncDecTokenizer('./EVA/src/bpe_dialog_new/vocab.txt')

CPU times: user 1min 18s, sys: 8.34 s, total: 1min 27s
Wall time: 27.5 s


In [5]:
%%time
ds = Dataset(batch_size=16)
dl = torch.utils.data.DataLoader(ds, num_workers=8, batch_size=None, pin_memory=True, prefetch_factor=10)

CPU times: user 1.62 s, sys: 887 ms, total: 2.5 s
Wall time: 2.52 s


In [10]:
%%time
fp16 = True
cuda = True

if fp16:
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=1e-3,
        momentum=0.9
    )
else:
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=1e-3,
        momentum=0.9
    )
if cuda:
    if fp16:
        model = model.half().cuda()
    else:
        model = model.cuda()

CPU times: user 1min 32s, sys: 41.9 s, total: 2min 14s
Wall time: 5.55 s


In [11]:
step = 0

In [12]:
from torch.nn import CrossEntropyLoss

In [13]:
optimizer.zero_grad()
losses = []
pbar = tqdm(dl)
loss_fct2 = CrossEntropyLoss(reduction='none')

for x, m0, y, m1, z in pbar:
    if cuda:
        x = x.cuda()
        y = y.cuda()
        z = z.cuda()
        m0 = m0.cuda()
        m1 = m1.cuda()
    with torch.cuda.amp.autocast():
        out = model(
            input_ids=x,
            attention_mask=m0,
            decoder_input_ids=y,
            decoder_attention_mask=m1,
            # labels=z
        )
        # loss = out.loss
        # loss = loss_fct(out.logits.view(-1, out.logits.size(-1)), z.view(-1))
        loss2 = loss_fct2(out.logits.view(-1, out.logits.size(-1)), z.view(-1))
        loss = torch.sum(loss2 * (z.view(-1) >= 0)) / torch.sum(z.view(-1) >= 0)

    loss.backward()
    if torch.isnan(loss) or torch.isinf(loss):
        print('bad loss')
        optimizer.zero_grad()
        continue
    else:
        optimizer.step()
        optimizer.zero_grad()
    loss = loss.detach().cpu().numpy()
    losses.append(loss)
    losses = losses[-100:]
    pbar.set_description(f'step: {step} loss: {np.mean(losses):.4f}')
    step += 1
    if step > 0 and step % (60 * 60) == 0:
        print('save', step, loss)
        torch.save(model.state_dict(), f'model_{step}.pt')
        torch.save(optimizer.state_dict(), f'opt_{step}.pt')

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...Building prefix dict from the default dictionary ...Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...Loading model from cache /tmp/jieba.cache


Loading model from cache /tmp/jieba.cache

Loading model from cache /tmp/jieba.cacheLoading model from cache /tmp/jieba.cacheLoading model from cache /tmp/jieba.cacheLoading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cacheLoading model from cache /tmp/jieba.cache





Loading model cost 0.836 seconds.Loading model cost 0.835 seconds.

Prefix dict has been built successfully.Prefix dict has been built successfully.Loading model cost 0.838 seconds.


Prefix dict has been built successfull

save 3600 1.8041242


step: 7199 loss: 1.5582: : 7199it [50:23,  2.30it/s]

save 7200 1.6094422


step: 10799 loss: 1.4074: : 10799it [1:15:44,  2.29it/s]

...

save 172800 1.0148427


step: 176399 loss: 1.0192: : 176399it [20:45:34,  2.40it/s]

save 176400 1.0250503


step: 176750 loss: 1.0093: : 176751it [20:48:24,  2.36it/s]


KeyboardInterrupt: 